> # This file has errors in field format and has been improved into another [file](../w-1/w-1.ipynb)

In [1]:
from uuid import uuid4
from pathlib import Path
import pandas as pd
import os, io, json, time, requests
from tqdm import trange, tqdm
from PIL import Image

In [2]:
df=pd.read_csv("https://xolyn.github.io/data/nysgdf.csv")
# imglist=sorted(os.listdir("D:\\OneDrive - Cornell University\\JselZJOB\\nysg\\w3\\img\\"), key=lambda x: int(x.split("-")[0]))
imglist=sorted(os.listdir("D:\\OneDrive - Cornell University\\JselZJOB\\nysg\\w3\\img\\"))


def getimgstr(id,prefix="",glue="|"):
    assert glue in ['<','>','|',':','?','\\','/','"','*'], "invalid, might cause ambiguity"
    return glue.join([prefix+x for x in imglist if x.split("-")[0]==str(id)])

def to12hr(timestr):
    time=timestr.split(":")
    if int(time[0])<12:
        return timestr+" am"
    elif int(time[0])==12:
        return timestr+" pm"
    else:
        if len(str(int(time[0])-12))==1:
            return "0"+str(int(time[0])-12)+":"+time[1]+" pm"
        else:
            return str(int(time[0])-12)+":"+time[1]+" pm"


df.image1=df["objectID"].apply(getimgstr)
df=df.fillna("")

#handle dates
df['photo_date'] = pd.to_datetime(df['photo_date'], format='%m/%d/%Y')
df['photo_date'] = df['photo_date'].dt.strftime('%Y-%m-%d')

#handle time
df['photo_time'] = df['photo_time'].apply(to12hr)
df
# df.to_csv("df_for_upload.csv",encoding="utf-8")

,objectID,creator,post_comment,est_depth,photo_time,alternative_loc,location_longitude,location_latitude,photo_date,guessSource,guessCause,image1
0,20,,,,12:00 pm,,-73.837982,40.657893,2020-05-07,,,20-20.jpg
1,21,,tide water coming to the street from storm dra...,,12:00 pm,,-73.837842,40.657755,2020-05-07,,,21-21.jpg
2,22,,,,00:30 am,Hamilton Beach,-73.829444,40.653456,2020-05-07,,,22-22.jpg
3,23,,6th road facing crossbay blvd,,00:39 am,,-73.817104,40.609023,2020-05-07,,,23-23.jpg
4,24,,,,00:30 am,,-73.829534,40.653500,2020-05-07,,,24-24.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
719,775,,Picture taken fro 102nd Street looking down 16...,5.0,09:31 am,"Howard Beach, Queens NY",-73.831090,40.658700,2023-02-21,,,775-1102.jpg
720,776,,,24.0,09:57 pm,East Elmhurst,-73.886160,40.765840,2023-04-30,,,776-1103.jpg
721,777,,This is located on rutland and Kingston Brookl...,12.0,06:30 pm,Brooklyn,-73.942715,40.659874,2023-06-02,"roads_streets,sidewalks,property,other. Homes","Rainfall_Water_pooling_from_dir,Storm_drain_Wa...",777-1104.jpg|777-1105.jpg
722,778,,,4.0,09:49 pm,,-73.942680,40.659922,2023-06-12,,,778-1106.jpg


In [4]:
def uploadsingle(objID=None, idx=None, user_id=9860, user_hash="$P$BQQLTdlS2/AO.60WWx4A9hfVAlR06o1"):
    '''
    Upload the img AND then report.\n\nFirst positional param. is the objectID, second is the index.\n\nIf you pass in two params., they have to refer to the same line!
    '''
    assert objID!=None or idx!=None, "either objectID or index has to be not None!"
    if objID!=None and idx!=None:
        assert list(df[df.objectID==objID].index)[0]==idx, "objectID and index input do not match!"
    if idx==None:
        idx=list(df[df.objectID==objID].index)[0] #works as objectID is PK
    if objID==None:
        objID=df.loc[idx,"objectID"]
    api_url = "https://mycoast.org/blueurchin-js/appthings.php"
    imgList=df.loc[idx, "image1"].split("|")
    if len(imgList)==1 and imgList[0]=="":
        image_ids=[""]
    else:
        image_ids = [str(uuid4()) for _ in range(len(imgList))] 
    
    imgzip=zip(imgList,image_ids)
    
    imgHeader = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0", "Accept": "*/*" }
    postHeader={ "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0", "Accept": "*/*", "Content-Type": "application/json" }
    
    # image upload (make sure all images are of .jpg, not just extension):
    if image_ids!=[""]:
        for img,imgid in imgzip:
            image_path=os.path.join("D:\\OneDrive - Cornell University\\JselZJOB\\nysg\\w3\\img",img)
            with open(image_path, 'rb') as file:
                file_content = file.read()
                response = requests.post(
                    f"{api_url}?apiv=5&user={user_id}&hash={user_hash}&upload=image&id={imgid}&purpose=0",
                    files = {'upfile': ('upfile', file_content)},headers=imgHeader
                )
            print("Image upload response:", response.text) 

    # form upload:

    form =[
        {"name":"_gform-form-id", "value": 92 },
        {"name":"from_device", "value": "Lingyu import-0522" },
        {"name":"photo_date", "value": df.loc[idx,"photo_date"]},
        {"name":"photo_time", "value": df.loc[idx,"photo_time"]},
        {"name":"image1", "value": image_ids },
        {"name":"guessCause", "value": df.loc[idx,"guessCause"] },
        {"name":"guessSource", "value": df.loc[idx,"guessCause"] },
        {"name":"guessDepth", "value": df.loc[idx,"est_depth"] },
        {"name":"post_comment", "value": df.loc[idx,"post_comment"]},
        {"name":"location_longitude", "value": df.loc[idx,"location_longitude"] },
        {"name":"location_latitude", "value": df.loc[idx,"location_latitude"]},
        {"name":"alternate-location-name", "value": df.loc[idx,"alternative_loc"]},
    ]
    print(objID,"-Form details:",form) #comment this out after test ok to avoid verbose output
    form=json.dumps(form)
    reportId=str(uuid4())
    form_response = requests.post(f"{api_url}?apiv=5&user={user_id}&hash={user_hash}&upload=report&id={reportId}", data=form, headers=postHeader)
    print("Form submission response:", form_response.text)

### testing:

In [116]:
# api_url = "https://mycoast.org/blueurchin-js/appthings.php"
# myCoastUserId = "123" 
## 123 is not a valid id
# myCoastUserHash = "123456789abcdef"
## also not a valid id
# image_path="D:\\OneDrive - Cornell University\\JselZJOB\\nysg\\w3\\img\\24-24.jpg" 
uploadsingle(idx=78)

Image upload response: {"status":"OK","result":""}
Form details: [{'name': '_gform-form-id', 'value': 92}, {'name': 'from_device', 'value': 'Lingyu import-0522'}, {'name': 'photo_date', 'value': '2020-10-27'}, {'name': 'photo_time', 'value': '02:27 pm'}, {'name': 'image1', 'value': ['40a1d229-428a-4200-a13d-092ee6e1a183']}, {'name': 'guessCause', 'value': ''}, {'name': 'guessSource', 'value': ''}, {'name': 'guessDepth', 'value': ''}, {'name': 'post_comment', 'value': ''}, {'name': 'location_longitude', 'value': -73.8387805341589}, {'name': 'location_latitude', 'value': 40.662639108405}]
Form submission response: {"status":"OK","result":""}


In [7]:
df.iloc[78]

objectID                      100
creator                          
post_comment                     
est_depth                        
photo_time               02:27 pm
alternative_loc                  
location_longitude     -73.838781
location_latitude       40.662639
photo_date             2020-10-27
guessSource                      
guessCause                       
image1                100-131.jpg
Name: 78, dtype: object

In [5]:
uploadsingle(objID=161)
uploadsingle(objID=724)
uploadsingle(objID=694)

Image upload response: {"status":"OK","result":""}
161 -Form details: [{'name': '_gform-form-id', 'value': 92}, {'name': 'from_device', 'value': 'Lingyu import-0522'}, {'name': 'photo_date', 'value': '2021-05-25'}, {'name': 'photo_time', 'value': '12:29 pm'}, {'name': 'image1', 'value': ['5b2041ef-6a42-4957-869d-050d9a4c9dd6']}, {'name': 'guessCause', 'value': 'Storm_drain_Water_bubbling_up_f,other. Manhole'}, {'name': 'guessSource', 'value': 'Storm_drain_Water_bubbling_up_f,other. Manhole'}, {'name': 'guessDepth', 'value': 10.0}, {'name': 'post_comment', 'value': ''}, {'name': 'location_longitude', 'value': -73.8073027739978}, {'name': 'location_latitude', 'value': 40.5951721690931}, {'name': 'alternate_location_name', 'value': ''}]
Form submission response: {"status":"OK","result":""}
Image upload response: {"status":"OK","result":""}
Image upload response: {"status":"OK","result":""}
Image upload response: {"status":"OK","result":""}
724 -Form details: [{'name': '_gform-form-id', 'v

In [51]:
Image.open(os.path.join("D:\\OneDrive - Cornell University\\JselZJOB\\nysg\\w3\\img\\",df[df.objectID==100]["image1"].values[0])).format

'JPEG'

# Bulk uploading:

In [51]:
def getImg(oid, prefix=""):
    img=list(df[df.objectID==oid]["image1"])[0]
    return [prefix+x for x in img.split("|")]

In [121]:
json.dumps([{"name": "_gform-form-id", "value": 176}, {"name": "from_device", "value": "Lingyu import 1"}, {"name": "photo_date", "value": "7/20/2021"}, {"name": "photo_time", "value": "18:17"}, {"name": "image1", "value": "d68e09ac-c204-40d8-acba-ede2e2cbc406"}, {"name": "guessCause", "value": ""}, {"name": "guessSource", "value": ""}, {"name": "guessDepth", "value": ""}, {"name": "post_comment", "value": "Flooded yesterday evening. "}, {"name": "location_longitude", "value": -73.7694660397022}, {"name": "location_latitude", "value": 40.6072411700354}]
)

'[{"name": "_gform-form-id", "value": 176}, {"name": "from_device", "value": "Lingyu import 1"}, {"name": "photo_date", "value": "7/20/2021"}, {"name": "photo_time", "value": "18:17"}, {"name": "image1", "value": "d68e09ac-c204-40d8-acba-ede2e2cbc406"}, {"name": "guessCause", "value": ""}, {"name": "guessSource", "value": ""}, {"name": "guessDepth", "value": ""}, {"name": "post_comment", "value": "Flooded yesterday evening. "}, {"name": "location_longitude", "value": -73.7694660397022}, {"name": "location_latitude", "value": 40.6072411700354}]'

In [55]:
getImg(48)

['48-56.jpg']

In [83]:
# start bulk uploading:
image_path="D:\\OneDrive - Cornell University\\JselZJOB\\nysg\\w3\\img\\"
t = trange(len(df), desc='')
for i in t:
    oid=df.loc[i,"objectID"]
    api_url = "https://mycoast.org/blueurchin-js/appthings.php"
    image_id = str(uuid4())

    imgHeader = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0", "Accept": "*/*" }
    postHeader={ "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0", "Accept": "*/*", "Content-Type": "application/json" }
    if image_path != None and image_path != "": 
        with open(image_path, 'rb') as file:
            file_content = file.read()
            upFiles = {'upfile': (f'{os.path.basename(image_path)}', file, 'image/jpeg')}

            response = requests.post(
                f"{api_url}?apiv=5&user={user_id}&hash={user_hash}&upload=image&id={image_id}&purpose=0",
                files = {'upfile': ('upfile', file_content)},headers=imgHeader
            )
        print("Image upload response:", response.text)

    form =[
        {"name":"_gform-form-id", "value": objectID },
        {"name":"from_device", "value": "Lingyu import 1" },
        {"name":"photo_date", "value": df.loc[objectID,"photo_date"]},
        {"name":"photo_time", "value": df.loc[objectID,"photo_time"]},
        {"name":"image1", "value": image_id },
        {"name":"guessCause", "value": df.loc[objectID,"guessCause"] },
        {"name":"guessSource", "value": df.loc[objectID,"guessCause"] },
        {"name":"guessDepth", "value": df.loc[objectID,"est_depth"] },
        {"name":"post_comment", "value": df.loc[objectID,"post_comment"]},
        {"name":"location_longitude", "value": df.loc[objectID,"location_longitude"] },
        {"name":"location_latitude", "value": df.loc[objectID,"location_latitude"]},
    ]

    form=json.dumps(form)

    reportId=str(uuid4())
    form_response = requests.post(f"{api_url}?apiv=5&user={user_id}&hash={user_hash}&upload=report&id={reportId}", data=form, headers=postHeader)
    print("Form submission response:", form_response.text)

    t.set_description("Report ID %i processed" % i)
    time.sleep(1)


Repost ID 723 processed: 100%|██████████| 724/724 [00:00<00:00, 2242.98it/s]
